
# Иванов Роман
# 98 Вариант
# Группа №9372
# Задача 3


На основе экспертных данных построена вероятностная модель диагностики определенной группы заболеваний (З1, З2, З3), эпидемиологическая обстановка по которым в указанное время приведена в табл. 2. Данные заболевания могут вызывать определенные симптомы (С1, С2, С3) с различными вероятностями (см. табл. 3), причем возникновение симптома при отсутствии заболевания маловероятно (0.001). Известно также, что у человека в определенный момент времени не может присутствовать более одного заболевания из этой группы.
Существует диагностическая процедура, которая позволяет выявить заболевание З3. Стоимость процедуры — 700 руб., а ее вероятностные характеристики приведены в табл. 4.
Известно два (взаимоисключающих) вида терапии Т1 и Т2 стоимостью 1200 руб. и 3400
руб., эффективность которых применительно к различным заболеваниям приведена в табл. 5.
Составьте диаграмму влияния и дерево решений для принятия решения о назначении анализа и выборе варианта терапии.

![title](2.png)

![title](3.png)

![title](4.png)

![title](5.png)

1. Определите стоимость информации о наличии у пациента каждого из симптомов.
2. При условии, что у пациента наблюдаются только симптом(ы) C1 и C2, определите:

(a) Вероятности того, что у пациента присутствует каждое из заболеваний. Какое является наиболее вероятным?

(b) Оптимальные действия по диагностике и лечению. Стоит ли проводить диагностическую процедуру?

(c) Определите «разумную» цену для проведения диагностической процедуры в данном случае.

In [174]:
import os

%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt

In [175]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

# *Создание байесовской сети*

In [176]:
medicine_bn = gum.BayesNet()
print(medicine_bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0}


# *Переменные (и вероятностные узлы)*

Переменная Disease (Болезнь) может принимать одно из четырех значений 'Нет', 'З1', 'З2' и 'З3'

In [177]:
# Создадим переменную 
# Первый параметр - имя переменной, второй - опциональный комментарий, а третий - количство значений
va = gum.LabelizedVariable('Disease', 'Is the patient sick?', 4)
va.changeLabel(0, 'No')
va.changeLabel(1, 'Disease1')
va.changeLabel(2, 'Disease2')
va.changeLabel(3, 'Disease3')
va

(pyAgrum.LabelizedVariable@00000158D2BDB3B0) Disease:Labelized({No|Disease1|Disease2|Disease3})

In [178]:
medicine_bn.add(va)

0

Аналогично создаются и лругие переменные

In [179]:
# Симптом 1
medicine_bn.add(gum.LabelizedVariable('Symptom1', 'Does the patient have the symptom 1?', 2))
# Симптом 2
medicine_bn.add(gum.LabelizedVariable('Symptom2', 'Does the patient have the symptom 2?', 2))
# Симптом 3
medicine_bn.add(gum.LabelizedVariable('Symptom3', 'Does the patient have the symptom 3?', 2))
# Тест
medicine_bn.add(gum.LabelizedVariable('Test', 'Diagnostic test result', 2))


4

## Дуги

Необходимо связать узлы, соответствующие созданным переменным в струкуру, отражающую их взаимосвязь:

In [180]:
# Наличие болезни влияет на проявление симптомов и на результат диагностики:
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom1'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom2'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Symptom3'))
medicine_bn.addArc(medicine_bn.idFromName('Disease'), medicine_bn.idFromName('Test'))

In [117]:
medicine_bn

FileNotFoundError: [WinError 2] "dot" not found in path.

(pyAgrum.BayesNet<double>@00000158D282CB40) BN{nodes: 5, arcs: 4, domainSize: 64, dim: 36}

## Таблицы условных вероятностей


In [118]:
medicine_bn.cpt(medicine_bn.idFromName('Disease')).fillWith([1-0.005-0.02-0.003, 0.005, 0.02, 0.003])
medicine_bn.cpt(medicine_bn.idFromName('Disease'))

(pyAgrum.Potential<double>@00000158D2B833F0) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.9720  | 0.0050  | 0.0200  | 0.0030  |

Узнаем, в каком порядке необходимо вписывать переменные

In [119]:
medicine_bn.cpt(medicine_bn.idFromName('Symptom1')).var_names

['Disease', 'Symptom1']

In [120]:
symptom1 = medicine_bn.idFromName('Symptom1')
symptom2 = medicine_bn.idFromName('Symptom2')
symptom3 = medicine_bn.idFromName('Symptom3')

medicine_bn.cpt(symptom1)[{'Disease': 'No'}] = [0.999, 0.001]   # два значения: для Symptom1=0 и Symptom1=1 соответственно
medicine_bn.cpt(symptom1)[{'Disease': 'Disease1'}] = [0.1, 0.9]
medicine_bn.cpt(symptom1)[{'Disease': 'Disease2'}] = [0.2, 0.8]
medicine_bn.cpt(symptom1)[{'Disease': 'Disease3'}] = [0.5, 0.5]


medicine_bn.cpt(symptom2)[{'Disease': 'No'}] = [0.999, 0.001]    # два значения: для Symptom2=0 и Symptom2=1 соответственно
medicine_bn.cpt(symptom2)[{'Disease': 'Disease1'}] = [0.4, 0.6]
medicine_bn.cpt(symptom2)[{'Disease': 'Disease2'}] = [0.2, 0.8]
medicine_bn.cpt(symptom2)[{'Disease': 'Disease3'}] = [0.2, 0.8]


medicine_bn.cpt(symptom3)[{'Disease': 'No'}] = [0.999, 0.001]    # два значения: для Symptom2=0 и Symptom2=1 соответственно
medicine_bn.cpt(symptom3)[{'Disease': 'Disease1'}] = [0.5, 0.5]
medicine_bn.cpt(symptom3)[{'Disease': 'Disease2'}] = [0.4, 0.6]
medicine_bn.cpt(symptom3)[{'Disease': 'Disease3'}] = [0.2, 0.8]


In [121]:
medicine_bn.cpt(symptom1)

(pyAgrum.Potential<double>@00000158D2B83030) 
      ||  Symptom1         |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9990  | 0.0010  |
Diseas|| 0.1000  | 0.9000  |
Diseas|| 0.2000  | 0.8000  |
Diseas|| 0.5000  | 0.5000  |

In [122]:
medicine_bn.cpt(symptom2)

(pyAgrum.Potential<double>@00000158D2B82F30) 
      ||  Symptom2         |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9990  | 0.0010  |
Diseas|| 0.4000  | 0.6000  |
Diseas|| 0.2000  | 0.8000  |
Diseas|| 0.2000  | 0.8000  |

In [123]:
medicine_bn.cpt(symptom3)

(pyAgrum.Potential<double>@00000158D2B83410) 
      ||  Symptom3         |
Diseas||0        |1        |
------||---------|---------|
No    || 0.9990  | 0.0010  |
Diseas|| 0.5000  | 0.5000  |
Diseas|| 0.4000  | 0.6000  |
Diseas|| 0.2000  | 0.8000  |

Аналогичную работу проделываем и с тестированием заболевания З3. Как известно, если пациент действительно болеет З3, то тест окажется положительным в 97% случаев. А если он не болеет З3, то тест все равно окажется положительным, но уже в 12% случаев. Опишем это

In [124]:
test = medicine_bn.idFromName('Test')

medicine_bn.cpt(test)[{'Disease': 'No'}] = [0.88, 0.12]   # два значения: для Test=0 (отр.) и Test=1 (пол.)
medicine_bn.cpt(test)[{'Disease': 'Disease1'}] = [0.88, 0.12] 
medicine_bn.cpt(test)[{'Disease': 'Disease2'}] = [0.88, 0.12] 
medicine_bn.cpt(test)[{'Disease': 'Disease3'}] = [0.03, 0.97]

medicine_bn.cpt(test)

(pyAgrum.Potential<double>@00000158D2B83230) 
      ||  Test             |
Diseas||0        |1        |
------||---------|---------|
No    || 0.8800  | 0.1200  |
Diseas|| 0.8800  | 0.1200  |
Diseas|| 0.8800  | 0.1200  |
Diseas|| 0.0300  | 0.9700  |

# 1 Задание. Определите стоимость информации о наличии у пациента каждого из симптомов.

In [125]:
d = medicine_bn.variable(0)   # Болезнь
s1 = medicine_bn.variable(1)  # Симптом 1
c = gum.LabelizedVariable('Treat', 'Treated?', 3) # Лечение
c.changeLabel(0, 'No')
c.changeLabel(1, 'T1')
c.changeLabel(2, 'T2')

utility_potential_s1 = gum.Potential().add(c).add(s1).add(d)

# Значение полезности по факту не зависит от наличия симптома 
# просто копируем значения из таблицы utility_potential
for disease in ['No', 'Disease1', 'Disease2', 'Disease3']:
    for s1 in [0, 1]:
        for t in ['No', 'T1', 'T2']:
            utility_potential_s1[{'Disease': disease, 'Symptom1': s1, 'Treat': t}] = utility_potential[{'Disease': disease, 'Treat': t}]
utility_potential_s1 

(pyAgrum.Potential<double>@00000158D2B834F0) 
             ||  Treat                      |
Sympto|Diseas||No       |T1       |T2       |
------|------||---------|---------|---------|
0     |No    || 2000.0000| 600.0000| -1900.0000|
1     |No    || 2000.0000| 600.0000| -1900.0000|
0     |Diseas|| 1100.0000| 400.0000| -2400.0000|
1     |Diseas|| 1100.0000| 400.0000| -2400.0000|
0     |Diseas|| 1400.0000| 400.0000| -1900.0000|
1     |Diseas|| 1400.0000| 400.0000| -1900.0000|
0     |Diseas|| 1000.0000| -300.0000| -1700.0000|
1     |Diseas|| 1000.0000| -300.0000| -1700.0000|

In [126]:
joint_proba_d_s1 = medicine_bn.cpt(medicine_bn.idFromName('Disease')) * medicine_bn.cpt(medicine_bn.idFromName('Symptom1'))

In [127]:
(utility_potential_s1 * joint_proba_d_s1).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D2B83530) 
      ||  Symptom1         |
Treat ||0        |1        |
------||---------|---------|
No    || 1949.7060| 30.7940 |
T1    || 583.9668| 8.3332  |
T2    || -1856.3032| -45.5968|

Заметим, что вне зависимости от того, имеется ли симптом С1, оптимальным решением будет не лечить. Следовательно, при наличии информации о симптоме С1 стратегия будет полностью совпадать со стратегией без этой информации, а значит и ожидаемая полезность не изменится. Таким образом, цена информации о симптоме С1 равна 0.

In [128]:
# Потенциал определяется на наборе переменных. В нашем случае их две - 
# Болезнь (используем ее определение из сети) и Лечение.
d = medicine_bn.variable(0)
c = gum.LabelizedVariable('Treat', 'Treated?', 3)
c.changeLabel(0, 'No')
c.changeLabel(1, 'T1')
c.changeLabel(2, 'T2')

utility_potential = gum.Potential().add(c).add(d)
utility_potential.fillWith([  2000,  600,  -1900, 
                             1100,   400,  -2400, 
                             1400,   400,  -1900,
                             1000,  -300,  -1700])

(pyAgrum.Potential<double>@00000158D2B83810) 
      ||  Treat                      |
Diseas||No       |T1       |T2       |
------||---------|---------|---------|
No    || 2000.0000| 600.0000| -1900.0000|
Diseas|| 1100.0000| 400.0000| -2400.0000|
Diseas|| 1400.0000| 400.0000| -1900.0000|
Diseas|| 1000.0000| -300.0000| -1700.0000|

In [129]:
s2 = medicine_bn.variable(2)  # Симптом 2
utility_potential_s2 = gum.Potential().add(c).add(s2).add(d)
# Значение полезности по факту не зависит от наличия симптома 
# просто копируем значения из таблицы utility_potential
for disease in ['No', 'Disease1', 'Disease2', 'Disease3']:
    for s2 in [0, 1]:
        for t in ['No', 'T1', 'T2']:
            utility_potential_s2[{'Disease': disease, 'Symptom2': s2, 'Treat': t}] = utility_potential[{'Disease': disease, 'Treat': t}]
utility_potential_s2            

(pyAgrum.Potential<double>@00000158D2B83BB0) 
             ||  Treat                      |
Sympto|Diseas||No       |T1       |T2       |
------|------||---------|---------|---------|
0     |No    || 2000.0000| 600.0000| -1900.0000|
1     |No    || 2000.0000| 600.0000| -1900.0000|
0     |Diseas|| 1100.0000| 400.0000| -2400.0000|
1     |Diseas|| 1100.0000| 400.0000| -2400.0000|
0     |Diseas|| 1400.0000| 400.0000| -1900.0000|
1     |Diseas|| 1400.0000| 400.0000| -1900.0000|
0     |Diseas|| 1000.0000| -300.0000| -1700.0000|
1     |Diseas|| 1000.0000| -300.0000| -1700.0000|

In [130]:
joint_proba_d_s2 = medicine_bn.cpt(medicine_bn.idFromName('Disease')) * medicine_bn.cpt(medicine_bn.idFromName('Symptom2'))

In [131]:
(utility_potential_s2 * joint_proba_d_s2).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D14B9F90) 
      ||  Symptom2         |
Treat ||0        |1        |
------||---------|---------|
No    || 1950.4560| 30.0440 |
T1    || 584.8368| 7.4632  |
T2    || -1858.3732| -43.5268|

In [132]:
s3 = medicine_bn.variable(3)  # Симптом 2
utility_potential_s3 = gum.Potential().add(c).add(s3).add(d)
# Значение полезности по факту не зависит от наличия симптома 
# просто копируем значения из таблицы utility_potential
for disease in ['No', 'Disease1', 'Disease2', 'Disease3']:
    for s3 in [0, 1]:
        for t in ['No', 'T1', 'T2']:
            utility_potential_s3[{'Disease': disease, 'Symptom3': s3, 'Treat': t}] = utility_potential[{'Disease': disease, 'Treat': t}]
utility_potential_s3   

(pyAgrum.Potential<double>@00000158D14B9FD0) 
             ||  Treat                      |
Sympto|Diseas||No       |T1       |T2       |
------|------||---------|---------|---------|
0     |No    || 2000.0000| 600.0000| -1900.0000|
1     |No    || 2000.0000| 600.0000| -1900.0000|
0     |Diseas|| 1100.0000| 400.0000| -2400.0000|
1     |Diseas|| 1100.0000| 400.0000| -2400.0000|
0     |Diseas|| 1400.0000| 400.0000| -1900.0000|
1     |Diseas|| 1400.0000| 400.0000| -1900.0000|
0     |Diseas|| 1000.0000| -300.0000| -1700.0000|
1     |Diseas|| 1000.0000| -300.0000| -1700.0000|

In [133]:
joint_proba_d_s3 = medicine_bn.cpt(medicine_bn.idFromName('Disease')) * medicine_bn.cpt(medicine_bn.idFromName('Symptom3'))

In [134]:
(utility_potential_s3 * joint_proba_d_s3).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D1889FF0) 
      ||  Symptom3         |
Treat ||0        |1        |
------||---------|---------|
No    || 1956.6060| 23.8940 |
T1    || 586.6368| 5.6632  |
T2    || -1867.1732| -34.7268|

Аналогичные выводы можно сделать и при симтомах С2 и С3. 
Т.е. При всех симптомах стоимость информации равна 0.

# 2 Задание

# 2.a Вероятности того, что у пациента присутствует каждое из заболеваний. Какое является наиболее вероятным?

Пусть для начала у пациента наблюдается только С1

In [135]:
ie=gum.VariableElimination(medicine_bn)
ie.eraseAllEvidence()
ie.setEvidence({'Symptom1': 1}) # указываем, что наблюдается только С1
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D2B83C50) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0423  | 0.1959  | 0.6965  | 0.0653  |

Видим, что если пациент имеет симпотом С1, то вероятности заболеть 

| Заболевание |Вероятность|
|-------------|-----------|
|  31         |   0.1959  | 
|  32         |   0.6965  |
|  32         |   0.0653  | 


Т.е. при С2 вероятностьзаболеть З2 наибольшая

Теперь пусть у пациента наблюдается только С2

In [136]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom2': 1}) # указываем, что наблюдается только С2
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D2B839B0) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0434  | 0.1341  | 0.7152  | 0.1073  |

Видим, что если пациент имеет симпотом С1, то вероятности заболеть 

| Заболевание |Вероятность|
|-------------|-----------|
|  31         |   0.1341  | 
|  32         |   0.7152  |
|  32         |   0.1073  | 

Видно, что при симптоме С2, вероятность заболеть З2 стала еще больше, вероятность 31 немного уменьшилась, а вероятность З3 немного поднялась

Теперь посмотрим на результаты при условии, что пациент имеет сразу оба симптома

In [137]:
ie.eraseAllEvidence()
# Пусть известно, что у пациента присутствуют оба симптома (про тест ничего не известно)
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D2B83970) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0001  | 0.1617  | 0.7664  | 0.0719  |

| Заболевание |Вероятность|
|-------------|-----------|
|  31         |   0.1617  | 
|  32         |   0.7664  |
|  32         |   0.0719  | 

По-прежнему З2 является наиболее вероятном. Можно сделать вывод, что З1 И З3 являются наиболее редкими заболеваниями.

#  2.b Оптимальные действия по диагностике и лечению. Стоит ли проводить диагностическую процедуру?

На предыдущем шаге мы выяснили, что при любом случаи вероятность заболевания **З2** выше заболевание **З1** и **З3**.Поэтому оптимальнее всего было бы не проводить диагностической процедуры.

Но можем заметить, что если наблюдается только  С2, то вероятности З1 и З2 практически равны, возможно тут нам и поможет диагностическая процедура.

In [138]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1,
               'Test': 0})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D188AA10) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0001  | 0.1737  | 0.8236  | 0.0026  |

если тест **отрицательный** и присутствуют оба симптома, то высока вероятность того, что имеется заболевание З2

In [139]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom1': 1,
                'Symptom2': 1,
               'Test': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D2B83670) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0000  | 0.1071  | 0.5079  | 0.3849  |

если тест **положительный** и присутствуют оба симптома, то хоть и вероятность З3 поднимается, но вероятность З2 по-прежнему наибольшая

In [140]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom1': 1,
               'Test': 0})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D14B9A70) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0452  | 0.2091  | 0.7434  | 0.0024  |

если тест **отрицательный** и присутствует только С1, то наиболее вероятно З2

In [141]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom1': 1,
               'Test': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D2B839D0) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0289  | 0.1339  | 0.4762  | 0.3609  |

если тест **положительный** и присутствует только С1, то наиболее вероятно З2, хотя вероятность З3 значительно выросло

In [142]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom2': 1,
               'Test': 0})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D14B9A50) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0485  | 0.1496  | 0.7979  | 0.0041  |

если тест **отрицательный** и присутствует только С2, то наиболее вероятно З2

In [143]:
ie.eraseAllEvidence()
ie.setEvidence({'Symptom2': 1,
               'Test': 1})
# Апостериорная вероятность того, что у пациента есть заболевание
ie.posterior('Disease')

(pyAgrum.Potential<double>@00000158D14B9C50) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.0247  | 0.0762  | 0.4064  | 0.4927  |

если тест **положительный** и присутствует только С2, то наиболее вероятно З3, хотя не сильно выше по сравнению с заболевание З2. 

Т.е. можем сделать выводы, что результаты теста либо подтвердят нам заболевание З2, при любой симптоматике С1 и С2. Либо мы не сможем с увереностью сказать, какой из заболевание присутствует 33 или З3. Но чтобы окончательно сделать выводы, перейдем к следующему этапу.

# Определение ожидаемой полезности на основе байесовской сети

Про эффект в условиии задачи ничего не говорится. Эффективность при заболеваниях 31-33, если не проводить терапию варьируется между 50-80. Будем считать, что эффективность человека в % и спосбоность его работать. Возьмем среднюю оплату труда в 60 т.р. Тогда один день будет стоить 2000р. Составим таблицу учитывая затраты на терапию. 



Подобная скаляризация позволяет записать функцию полезности следующим образом (колонка "Всего" показывает суммарную стоимость с учетом стоимости терапии - 1200 руб. и 3400 руб.)"


| Болезнь | Терапия  |  Эффект  | Эффект (руб.) |    Всего (руб.) |
|---------|----------|----------|---------------|-----------------|
|  Нет    |  Нет     |   100    |   2000        |    2000         | 
|  Нет    |  Т1      |    90    |   1800        |     600        |
|  Нет    |  Т2      |    75    |   1500        |   -1900        |
|  З1     |  Нет     |    55    |   1100        |    1100        |
|  З1     |  T1      |    80    |   1600        |     400        |
|  З1     |  T2      |    50    |   1000        |   -2400        |
|  З2     |  Нет     |    70    |   1400        |   1400        |
|  З2     |  T1      |    80    |   1600        |   400        |
|  З2     |  T2      |    75    |   1500        |   -1900        | 
|  З3     |  Нет     |    50    |   1000        |   1000        |
|  З3     |  Т1      |    45    |   900         |   -300        |
|  З3     |  Т2      |    85    |   1700        |   -1700        |


In [144]:
# Потенциал определяется на наборе переменных. В нашем случае их две - 
# Болезнь (используем ее определение из сети) и Лечение.
d = medicine_bn.variable(0)
c = gum.LabelizedVariable('Treat', 'Treated?', 3)
c.changeLabel(0, 'No')
c.changeLabel(1, 'T1')
c.changeLabel(2, 'T2')

utility_potential = gum.Potential().add(c).add(d)
utility_potential.fillWith([  2000,  600,  -1900, 
                             1100,   400,  -2400, 
                             1400,   400,  -1900,
                             1000,  -300,  -1700])

(pyAgrum.Potential<double>@00000158D2B83690) 
      ||  Treat                      |
Diseas||No       |T1       |T2       |
------||---------|---------|---------|
No    || 2000.0000| 600.0000| -1900.0000|
Diseas|| 1100.0000| 400.0000| -2400.0000|
Diseas|| 1400.0000| 400.0000| -1900.0000|
Diseas|| 1000.0000| -300.0000| -1700.0000|

In [145]:
def disease_proba_factor(ie, ev):
    ie.eraseAllEvidence()
    ie.setEvidence(ev)
    return ie.posterior('Disease') #.toarray()

In [146]:
disease_proba_factor(ie, {})

(pyAgrum.Potential<double>@00000158D2B83D10) 
  Disease                              |
No       |Disease1 |Disease2 |Disease3 |
---------|---------|---------|---------|
 0.9720  | 0.0050  | 0.0200  | 0.0030  |

In [147]:
# Произведение потенциалов эквивалентно произведению всех значений, у которых
# одинаковы значения соответствующих переменных (Disease),
# а margSumOut() осуществляет маржинализацию переменной Disease (сложение
# ячеек, соответствующих этой переменной для каждого сочетания других переменных)
(utility_potential * disease_proba_factor(ie, {})).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D2B83C70) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 1980.5000| 592.3000| -1901.9000|

По результатам можем сделать вывод о том, что оптимальным решением будет не лечить. А итоговая полезность будет составлять 
1980.5000

In [148]:

def max_u(potential):
    min_val = potential.min()
    if min_val < 0:
        potential = potential - (min_val - 1)
        return potential.max() + (min_val - 1)
    return potential.max()

def max_argmax_u(potential):
    min_val = potential.min()
    if min_val < 0:
        potential = potential - (min_val - 1)
        return potential.max() + (min_val - 1), potential.argmax()
    return potential.max(), potential.argmax()

def eu(ie, evidence, utility_potential):
    tmp = (utility_potential * disease_proba_factor(ie, evidence)).margSumOut(['Disease'])
    return max_argmax_u(tmp)

eu(ie, {}, utility_potential)

(1980.4999999999998, ([{'Treat': 0}], 3883.3999999999996))

Оценим распределение вероятностей результата теста при имеющейся информации и возможные исходы (с учетом теста и без него):

    1.1 Если мы не делаем тест и присутствует только симптом С2

In [149]:
(utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1})).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D2BCE690) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 1402.2248| 286.1239| -1884.1299|

Лечить не будем. Итоговая полезность 1402.2248

    1.2 Если мы не делаем тест и присутствует только симптом С1

In [150]:
(utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 0})).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D2BCE410) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 1387.6602| 397.4814| -2033.9493|

Лечить не будем. Итоговая полезность 1387.6602

    1.3. Если мы не делаемм тест и присутствуют оба симптома

In [151]:
(utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 1})).margSumOut(['Disease'])

(pyAgrum.Potential<double>@00000158D2BCE390) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 1322.7939| 349.7152| -1966.4632|

Лечить не будем. Итоговая полезность 1322.7939

In [152]:
def test_proba_factor(ie, ev):
    """Вычисление вероятности показаний теста в зависимости от имеющейся информации."""
    ie.eraseAllEvidence()
    ie.setEvidence(ev)
    return ie.posterior('Test') #.toarray()

    2.1. Если мы делаем тест и присутствует только симптом С2

In [153]:
# Если тест делаем, то возможны два исхода (не забываем учесть стоимость теста - 700 руб.): 
# 1. Тест окажется положительным
test_positive01 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1, 'Test': 1})).margSumOut(['Disease']) - 700 
test_positive01

(pyAgrum.Potential<double>@00000158D2BCDEF0) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 460.9642| -765.4422| -2473.6859|

Лечить не будем. Итоговая полезность 460.9642

    2.2 Если мы делаем тест и присутствует только симптом С1


In [154]:
test_positive10 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 0, 'Test': 1})).margSumOut(['Disease']) - 700 
test_positive10

(pyAgrum.Potential<double>@00000158D2BCE630) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 589.5450| -479.0483| -2649.4281|

Лечить не будем. Итоговая полезность 589.5450


    2.3 Если мы делаем тест и присутствует и симпотом С1 и С2

In [155]:
test_positive11 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 1, 'Test': 1})).margSumOut(['Disease']) - 700 
test_positive11

(pyAgrum.Potential<double>@00000158D2BCE4F0) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 513.9192| -569.4263| -2576.5882|

Лечить не будем. Итоговая полезность 513.9192

Но так же тест может оказаться и отрицательным

In [156]:
test_negative01 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1, 'Test': 0})).margSumOut(['Disease']) - 700
test_negative01

(pyAgrum.Potential<double>@00000158D2BCE1D0) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 805.5540| -263.3042| -2631.4318|

Лечить не будем. Итоговая полезность 805.5540

In [166]:
test_negative10 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 0, 'Test': 0})).margSumOut(['Disease']) - 700
test_negative10

(pyAgrum.Potential<double>@00000158D2BCE550) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 706.4411| -268.7279| -2750.1281|

Лечить не будем. Итоговая полезность 706.4411

In [167]:
test_negative11 = (utility_potential * disease_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 1, 'Test': 0})).margSumOut(['Disease']) - 700
test_negative11

(pyAgrum.Potential<double>@00000158D2BCE3F0) 
  Treat                      |
No       |T1       |T2       |
---------|---------|---------|
 646.8674| -301.8300| -2686.3356|

Лечить не будем. Итоговая полезность 646.8674

In [168]:
tmp01 = test_proba_factor(ie, {'Symptom1': 0, 'Symptom2': 1})
tmp01

(pyAgrum.Potential<double>@00000158D2BCE370) 
  Test             |
0        |1        |
---------|---------|
 0.7001  | 0.2999  |

## Таким образом, итоговая ожидаемая полезность при симптоме С2

In [169]:
tmp01[0]*max_u(test_negative01) + tmp01[1]*max_u(test_positive01)

702.2247818208621

In [170]:
tmp10 = test_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 0})
tmp10

(pyAgrum.Potential<double>@00000158D2BCDF10) 
  Test             |
0        |1        |
---------|---------|
 0.8393  | 0.1607  |

## Таким образом, итоговая ожидаемая полезность при симптоме C1

In [171]:
tmp10[0]*max_u(test_negative10) + tmp10[1]*max_u(test_positive10)

687.6602049935035

In [172]:
tmp11 = test_proba_factor(ie, {'Symptom1': 1, 'Symptom2': 1})
tmp11

(pyAgrum.Potential<double>@00000158D2BCE5D0) 
  Test             |
0        |1        |
---------|---------|
 0.8189  | 0.1811  |

## Таким образом, итоговая ожидаемая полезность при симптомах С1 и С2

In [173]:
tmp11[0]*max_u(test_negative11) + tmp11[1]*max_u(test_positive11)

622.7939068456618

Ожидаемая полезность с учетом знания результатов теста оказалась ниже во всех случая симпоматики С1 и С2, чем без учета знания результатов теста. Следовательно, тест в такой ситуации проводить нецелесообразно.

# 2.c Определите «разумную» цену для проведения диагностической процедуры в данном случае.

Сделаем итоговую таблицу:

|Симпотомы|Полезность БЕЗ теста|Полезность С тестом|Разница |
|---------|--------------------|-------------------|--------|
| C1      |   1387.660         |   687.660         |700     |
| C2      |   1402.225         |   702.225         |700     |
| C1 и С2 |   1322.794         |   622.794         |700     |
    

По данным результатм нельзя выставить разумную цену, так как разница между тестированием и без составляет 700